# Modeling Session 1
This set of exercises are meant to cover many of the topics from Day 1, with callbacks from Opti 101, 201, 202 along the way. You can work along in this notebook, but if you really just want to see the solutions, those can also be found in the GitHub repo.

In [51]:
%pip -q install gurobipy==13.0.0

Note: you may need to restart the kernel to use updated packages.


In [52]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

## Building Trust in Optimization

### Solution pools

In this section, we'll explore solution pools to build trust in a discrete model by
surfacing *multiple, near‑optimal* operating plans. We'll use the same storage + solar
dispatch context from earlier (charging/discharging a battery, buying from the grid, meeting
demand), but we’ll ask Gurobi to return a *set* of high‑quality alternatives.

Goal: Collect the top‑k solutions within a small optimality gap and compare them on
key features (objective, total grid energy, number of charge/discharge switches).

Note that some of this (the problem, and the approaches) should seem familiar...

#### 🔋 Energy Storage Optimization Model

##### Sets
$$
\begin{aligned}
T = \{0, 1, \dots, 11\} \quad \text{(time periods, e.g., hours)}
\end{aligned}
$$

##### Parameters
$$
\begin{aligned}
p_t & : \text{Electricity price in period } t \; [€/MWh] \\
d_t & : \text{Energy demand in period } t \; [\text{MWh}] \\
\eta_{\text{in}}, \eta_{\text{out}} & : \text{Charging and discharging efficiencies} \\
\text{cap} & : \text{Battery capacity } [\text{MWh}] \\
p_{\max} & : \text{Maximum charge/discharge power } [\text{MW}] \\
s_0 & : \text{Initial state of charge } [\text{MWh}]
\end{aligned}
$$

##### Decision Variables
$$
\begin{aligned}
ch_t &\ge 0 && \text{Charging power (MW)} \\
dis_t &\ge 0 && \text{Discharging power (MW)} \\
grid_t &\ge 0 && \text{Power purchased from the grid (MW)} \\
0 \le soc_t &\le \text{cap} && \text{State of charge (MWh)} \\
y_{ch,t} &\in \{0,1\} && \text{1 if charging in period } t \\
y_{dis,t} &\in \{0,1\} && \text{1 if discharging in period } t
\end{aligned}
$$

##### Objective Function
Minimize total grid cost and a small switching penalty:
$$
\begin{aligned}
\min \; Z = \sum_{t \in T} p_t \cdot grid_t \;+\; 10^{-3} \sum_{t \in T} (y_{ch,t} + y_{dis,t})
\end{aligned}
$$


##### Constraints

1. Mutual Exclusivity
$$
\begin{aligned}
y_{ch,t} + y_{dis,t} \le 1 \quad \forall t \in T
\end{aligned}
$$

2. Power Limits
$$
\begin{aligned}
ch_t &\le p_{\max} \, y_{ch,t} \\
dis_t &\le p_{\max} \, y_{dis,t} \quad \forall t \in T
\end{aligned}
$$

3. Energy Balance
$$
\begin{aligned}
grid_t + \eta_{\text{out}} \, dis_t \ge d_t + ch_t \quad \forall t \in T
\end{aligned}
$$

4. State-of-Charge (SoC) Dynamics
$$
\begin{aligned}
soc_0 &= s_0 + \eta_{\text{in}} \, ch_0 - \frac{1}{\eta_{\text{out}}} \, dis_0 \\
soc_t &= soc_{t-1} + \eta_{\text{in}} \, ch_t - \frac{1}{\eta_{\text{out}}} \, dis_t \quad \forall t > 0 \\
&0 \le soc_t \le \text{cap} \quad \forall t \in T
\end{aligned}
$$

For reference, here is our [solution pool documentation](https://docs.gurobi.com/projects/optimizer/en/current/features/solutionpool.html).

In [53]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Tiny instance (kept small so we can enumerate several pool solutions quickly)
T = range(12)                        # time periods
price = [20,18,16,15,16,18,25,40,45,35,28,22]  # grid price €/MWh
demand = [5,5,5,5,6,6,7,7,7,6,6,5]  # MWh demand each period

eta_in, eta_out = 0.95, 0.95        # round-trip efficiency
cap = 10.0                          # MWh battery capacity
pmax = 4.0                          # MW charge/discharge limit
s0 = 2.0                            # initial state of charge

m = gp.Model("storage_pool")

# Decision variables
ch = m.addVars(T, name="charge", lb=0, ub=pmax)
dis = m.addVars(T, name="discharge", lb=0, ub=pmax)
grid = m.addVars(T, name="grid", lb=0)
soc = m.addVars(T, name="soc", lb=0, ub=cap)
# on/off switching to discourage rapid toggling (makes model MIP)
y_ch = m.addVars(T, vtype=GRB.BINARY, name="y_ch")
y_dis = m.addVars(T, vtype=GRB.BINARY, name="y_dis")

# Logic: can’t charge and discharge simultaneously
m.addConstrs((y_ch[t] + y_dis[t] <= 1 for t in T), name="no_simul")
m.addConstrs((ch[t] <= pmax * y_ch[t] for t in T), name="charge_gate")
m.addConstrs((dis[t] <= pmax * y_dis[t] for t in T), name="discharge_gate")

# Energy balance vs demand
m.addConstrs((grid[t] + dis[t]*eta_out >= demand[t] + ch[t] for t in T), name="balance")

# State of charge dynamics
for t in T:
    if t == 0:
        m.addConstr(soc[t] == s0 + ch[t]*eta_in - dis[t]/eta_out, name=f"soc_{t}")
    else:
        m.addConstr(soc[t] == soc[t-1] + ch[t]*eta_in - dis[t]/eta_out, name=f"soc_{t}")

# Mild cycling preference: penalize on/off changes to diversify plans
switches = gp.quicksum((y_ch[t] + y_dis[t]) for t in T)

# Cost: energy purchased from grid
cost = gp.quicksum(price[t]*grid[t] for t in T)

# Multi-criteria via a single objective (lexico done elsewhere; here we pool around cost)
m.setObjective(cost + 1e-3 * switches, GRB.MINIMIZE)

# Ask for a pool of near‑optimal solutions
m.setParam(GRB.Param.PoolSearchMode, 2)  # systematic search for k‑best
m.setParam(GRB.Param.PoolSolutions, 500)  # keep up to 500
m.setParam(GRB.Param.PoolGap, 0.02)      # within 2% of best found
m.optimize()

# Collect solutions
rows = []
for i in range(m.SolCount):
    m.setParam(GRB.Param.SolutionNumber, i)
    obj = m.PoolObjVal
    total_grid = sum(grid[t].Xn for t in T)
    total_switch = sum(y_ch[t].Xn + y_dis[t].Xn for t in T)
    rows.append({"obj": obj, "grid_MWh": total_grid, "switch_count": total_switch})

sols = pd.DataFrame(rows).sort_values("obj").reset_index(drop=True)

best_obj = m.ObjVal
sols["rel_diff_to_best_pct"] = ((sols["obj"] - best_obj) / abs(best_obj)* 100).round(6).astype(str) + "%"
sols_unique = sols.drop_duplicates().reset_index(drop=True)
sols_unique


Set parameter PoolSearchMode to value 2
Set parameter PoolSolutions to value 500
Set parameter PoolGap to value 0.02
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[rosetta2] - Darwin 24.6.0 24G325)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
PoolSolutions  500
PoolSearchMode  2
PoolGap  0.02

Optimize a model with 60 rows, 72 columns and 155 nonzeros (Min)
Model fingerprint: 0xd2c985b1
Model has 36 linear objective coefficients
Variable types: 48 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [9e-01, 4e+00]
  Objective range  [1e-03, 4e+01]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 1770.9009999
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 59 rows, 71 columns, 153 nonzeros
Variable types: 47 continuous, 24 integer (24 binary)

Root relaxation: objective 1.562452e+03, 48 iterations, 

,obj,grid_MWh,switch_count,rel_diff_to_best_pct
0,1562.453053,69.696316,7.0,0.0%
1,1562.454053,69.696316,8.0,6.4e-05%
2,1562.455053,69.696316,9.0,0.000128%
3,1562.456053,69.696316,10.0,0.000192%
4,1562.457053,69.696316,11.0,0.000256%
5,1562.458053,69.696316,12.0,0.00032%
6,1564.867842,69.396053,6.0,0.154551%
7,1564.868842,69.396053,7.0,0.154615%
8,1564.869842,69.396053,8.0,0.154679%
9,1564.870842,69.396053,9.0,0.154743%


<div style="
  border-left: 6px solid #f59e0b;  /* amber/yellow */
  padding:14px 16px;
  border-radius:10px;
  margin:12px 0;
  font-size:16px; line-height:1.4;">
  <strong style="font-size:18px;">💬 Explainability Moment</strong><br>
  <em>Pause here.</em> How would you explain the methodology of solution pools and this group of solutions to a <b>non-technical stakeholder</b>? 
</div>


### Multiple scenarios

In this section, we'll evaluate multiple scenarios by changing the *demand* and *price*
profiles and re‑solving the same dispatch model. This mirrors the “what‑if” analysis you saw
earlier: same structure, new data.

Goal: Run three scenarios (Base, High‑Price Peak, Evening‑Heavy Demand) and compare total
grid cost and final state of charge.

In [54]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

T = range(12)
base_price = [20,18,16,15,16,18,25,40,45,35,28,22]
base_demand = [5,5,5,5,6,6,7,7,7,6,6,5]

scenarios = {
    "Base": (base_price, base_demand),
    "HighPricePeak": ([p*1.2 if 7<=t<=9 else p for t,p in enumerate(base_price)], base_demand),
    "EveningHeavyDemand": (base_price, [d*1.2 if t>=7 else d for t,d in enumerate(base_demand)]),
}

eta_in, eta_out = 0.95, 0.95
cap, pmax, s0 = 10.0, 4.0, 2.0

def solve(price, demand):
    m = gp.Model("storage_scen")
    ch = m.addVars(T, name="charge", lb=0, ub=pmax)
    dis = m.addVars(T, name="discharge", lb=0, ub=pmax)
    grid = m.addVars(T, name="grid", lb=0)
    soc = m.addVars(T, name="soc", lb=0, ub=cap)

    # LP version to keep focus on param changes
    m.addConstrs((grid[t] + dis[t]*eta_out >= demand[t] + ch[t] for t in T), name="balance")
    for t in T:
        if t == 0:
            m.addConstr(soc[t] == s0 + ch[t]*eta_in - dis[t]/eta_out)
        else:
            m.addConstr(soc[t] == soc[t-1] + ch[t]*eta_in - dis[t]/eta_out)

    cost = gp.quicksum(price[t]*grid[t] for t in T)
    m.setObjective(cost, GRB.MINIMIZE)
    m.setParam("OutputFlag", 0)
    m.optimize()

    return float(cost.getValue()), float(soc[max(T)].X)

rows = []
for name, (p, d) in scenarios.items():
    total_cost, final_soc = solve(p, d)
    rows.append({"scenario": name, "total_cost": round(total_cost,2), "final_soc": round(final_soc,2)})

pd.DataFrame(rows).sort_values("total_cost")

,scenario,total_cost,final_soc
0,Base,1562.45,0.0
1,HighPricePeak,1648.87,0.0
2,EveningHeavyDemand,1779.05,0.0


<div style="
  border-left: 6px solid #f59e0b;  /* amber/yellow */
  padding:14px 16px;
  border-radius:10px;
  margin:12px 0;
  font-size:16px; line-height:1.4;">
  <strong style="font-size:18px;">💬 Explainability Moment</strong><br>
  <em>Pause here.</em> How do you think running multiple scenarios can build trust in your optimization model? What other scenarios do you think could be helpful to run for this problem?
</div>


<div style="
  border-left: 6px solid #3b82f6;  /* blue */
  padding:14px 16px;
  border-radius:10px;
  margin:12px 0;
  font-size:16px; line-height:1.4;">
  <strong style="font-size:18px;">📘 Optional Activity: Back to Hidden Gems</strong><br>
  Using Gurobi's multi-scenario functionality to solve the same problem above. 
</div>


### Pi values and sensitivity analysis

In this section, we'll inspect dual values (π) and sensitivity for the *LP* version
of the dispatch model. The dual (shadow) price on a constraint tells you how much the
objective would improve per unit relaxation of that constraint’s RHS - holding the basis fixed.

Goal: Solve the LP, read π for the balance constraints, and identify which periods
are most “binding” (largest |π|). Then, perturb demand slightly and see if the objective
change agrees with π.

In [55]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

T = range(12)
price = [20,18,16,15,16,18,25,40,45,35,28,22]
demand = [5,5,5,5,6,6,7,7,7,6,6,5]

eta_in, eta_out = 0.95, 0.95
cap, pmax, s0 = 10.0, 4.0, 2.0

# Build LP (no binaries)
m = gp.Model("storage_duals")
ch = m.addVars(T, name="charge", lb=0, ub=pmax)
dis = m.addVars(T, name="discharge", lb=0, ub=pmax)
grid = m.addVars(T, name="grid", lb=0)
soc = m.addVars(T, name="soc", lb=0, ub=cap)

balance = m.addConstrs((grid[t] + dis[t]*eta_out >= demand[t] + ch[t] for t in T), name="balance")
for t in T:
    if t == 0:
        m.addConstr(soc[t] == s0 + ch[t]*eta_in - dis[t]/eta_out, name=f"soc_{t}")
    else:
        m.addConstr(soc[t] == soc[t-1] + ch[t]*eta_in - dis[t]/eta_out, name=f"soc_{t}")

cost = gp.quicksum(price[t]*grid[t] for t in T)
m.setObjective(cost, GRB.MINIMIZE)
m.setParam("OutputFlag", 0)
m.optimize()

# Read duals (Pi) and slacks
rows = []
for t in T:
    con = balance[t]
    rows.append({
        "t": t,
        "pi": con.Pi,
        "slack": con.Slack,
    })

pi_df = pd.DataFrame(rows).sort_values("pi", ascending=False)
print("Top periods by shadow price (higher pi => relaxing balance helps more):")
display(pi_df.head(6))

# Sensitivity check: bump demand at period with largest pi by +0.1 and compare objective change
t_star = int(pi_df.iloc[0]["t"])
bump = 0.1

def solve_with_demand(d):
    m2 = gp.Model("storage_duals_copy")
    ch2 = m2.addVars(T, name="charge", lb=0, ub=pmax)
    dis2 = m2.addVars(T, name="discharge", lb=0, ub=pmax)
    grid2 = m2.addVars(T, name="grid", lb=0)
    soc2 = m2.addVars(T, name="soc", lb=0, ub=cap)
    bal2 = m2.addConstrs((grid2[t] + dis2[t]*eta_out >= d[t] + ch2[t] for t in T))
    for t in T:
        if t == 0:
            m2.addConstr(soc2[t] == s0 + ch2[t]*eta_in - dis2[t]/eta_out)
        else:
            m2.addConstr(soc2[t] == soc2[t-1] + ch2[t]*eta_in - dis2[t]/eta_out)
    cost2 = gp.quicksum(price[t]*grid2[t] for t in T)
    m2.setObjective(cost2, GRB.MINIMIZE)
    m2.setParam("OutputFlag", 0)
    m2.optimize()
    return float(cost2.getValue())

base_obj = float(cost.getValue())
d2 = demand.copy()
d2[t_star] += bump
new_obj = solve_with_demand(d2)

approx_change = pi_df.iloc[0]["pi"] * bump
actual_change = new_obj - base_obj

summary = pd.DataFrame({
    "t*":[t_star],
    "pi":[pi_df.iloc[0]["pi"]],
    "bump":[bump],
    "approx_obj_change (pi*bump)":[approx_change],
    "actual_obj_change":[actual_change],
})
display(summary)
 
# FYI explanation printout, commas only
print(
    f"\nFYI,\n"
    f"- Period {t_star} had the highest shadow price (pi = {pi_df.iloc[0]['pi']:.1f}), meaning extra demand there is most expensive.\n"
    f"- The bump of {bump} MWh is small enough that the linear, pi-based prediction is valid.\n"
    f"- Predicted vs actual change ({approx_change:.2f} vs {actual_change:.2f}) are nearly identical, so we are within the linear sensitivity range.\n"
    f"- Slack = 0 confirms this constraint was binding.\n"
    f"- In plain terms, adding 0.1 MWh at hour {t_star} costs about €{approx_change:.1f}, exactly as the model’s dual value predicted."
)

Top periods by shadow price (higher pi => relaxing balance helps more):


,t,pi,slack
8,8,45.0,0.0
7,7,40.0,0.0
9,9,35.0,0.0
10,10,28.0,0.0
6,6,25.0,0.0
11,11,22.0,0.0


,t*,pi,bump,approx_obj_change (pi*bump),actual_obj_change
0,8,45.0,0.1,4.5,4.5



FYI,
- Period 8 had the highest shadow price (pi = 45.0), meaning extra demand there is most expensive.
- The bump of 0.1 MWh is small enough that the linear, pi-based prediction is valid.
- Predicted vs actual change (4.50 vs 4.50) are nearly identical, so we are within the linear sensitivity range.
- Slack = 0 confirms this constraint was binding.
- In plain terms, adding 0.1 MWh at hour 8 costs about €4.5, exactly as the model’s dual value predicted.


### Cautionary Tale: When Duals Lie to You
 
Shadow prices (π’s) are powerful, but they only tell the truth *locally*, right around the current optimal solution, assuming nothing else changes.
 
Imagine this:
 
> The model says π = €45 at hour 8,  
> you say “Adding 1 MWh will cost €45!”,  
> you rerun the model, and the cost jumps by €70.
 
**What happened?**
 
- You crossed a kink in the feasible region, a new constraint became binding, so the old π no longer applies.  
- You took too large a step, duals are like instantaneous slopes, not good for big jumps.  
- You have integer logic (on/off decisions), duals only exist for the continuous relaxation, not for the real MIP behavior.  
- Or your scaling or units are inconsistent, a π in kWh means something different than in MWh.

<div style="
  border-left: 6px solid #f59e0b;  /* amber/yellow */
  padding:14px 16px;
  border-radius:10px;
  margin:12px 0;
  font-size:16px; line-height:1.4;">
  <strong style="font-size:18px;">💬 Moral of the story</strong><br>
Duals are great for intuition and small “what-if” nudges, but risky for big decisions. However, they are like weather forecasts, useful for local conditions, unreliable once you cross into another climate.
</div>


## Optimization's Impact in Industry: Finance

### Add a cardinality constraint to the basic Markowitz Portfolio Optimization model
Now we'll apply some of the concepts of *discrete decisions* from the session on Optimization in Finance. A great place to start is by testing out the OptiMod for [Mean-Variance Portfolio Optimization](https://gurobi-optimods.readthedocs.io/en/stable/index.html). Then, we'll build our own version of the same model to compare. Finally, we'll change and add to that model, similar to what's shown in the [Gurobi Finance](https://gurobi-finance.readthedocs.io/en/latest/index.html) documentation, to reflect different scenarios.

In [56]:
# Install the optimods package
%pip -q install gurobi-optimods
import numpy as np

from gurobi_optimods.datasets import load_portfolio
from gurobi_optimods.portfolio import MeanVariancePortfolio

Note: you may need to restart the kernel to use updated packages.


All of the individual OptiMods come with data to be used right out of the box for each pre-built model. Let's load that and briefly see what form the data is in. If you're using an OptiMod, pay attention to the format of the data to make sure any new data is in the same form.

In [57]:
np.set_printoptions(legacy='1.21')

data = load_portfolio()
cov_matrix = data.cov()
mu = data.mean()

In [58]:
# Show the expected returns for our potential 'assets'
mu

AA    0.001602
BB    0.014219
CC   -0.013629
DD    0.007160
EE    0.001796
FF   -0.004670
GG    0.002771
HH    0.005117
II   -0.000745
JJ    0.004155
dtype: float64

In [59]:
# Show the covariance matrix
cov_matrix

,AA,BB,CC,DD,EE,FF,GG,HH,II,JJ
AA,0.000256,0.000199,0.000077,0.000069,0.000145,0.000152,0.000160,0.000082,0.000086,0.000242
BB,0.000199,0.002965,0.001186,0.000650,0.001006,0.000736,0.000586,0.000154,0.000309,0.001625
CC,0.000077,0.001186,0.002303,0.000310,0.000776,0.000767,0.000405,0.000156,0.000132,0.000749
DD,0.000069,0.000650,0.000310,0.000591,0.000381,0.000255,0.000195,0.000075,0.000213,0.000445
EE,0.000145,0.001006,0.000776,0.000381,0.001424,0.000507,0.000331,-0.000007,0.000282,0.000753
FF,0.000152,0.000736,0.000767,0.000255,0.000507,0.000903,0.000270,0.000046,0.000112,0.000584
GG,0.000160,0.000586,0.000405,0.000195,0.000331,0.000270,0.000528,0.000101,0.000181,0.000368
HH,0.000082,0.000154,0.000156,0.000075,-0.000007,0.000046,0.000101,0.000581,-0.000032,-0.000068
II,0.000086,0.000309,0.000132,0.000213,0.000282,0.000112,0.000181,-0.000032,0.000659,0.000335
JJ,0.000242,0.001625,0.000749,0.000445,0.000753,0.000584,0.000368,-0.000068,0.000335,0.001644


The *cardinality constraint* we will implement first will just limit the number of positions we can take (i.e. the number of assets to invest in), which we'll initially set to 3. We also see the risk-aversion parameter, `gamma`($\gamma$). The number used for these examples is not specifically set, so feel free to change it. 

In [60]:
# We can define this risk-aversion coefficient
gamma = 20
# Max positions K
K = 3

# Fill in the above info for expected returns and the covariance, respectively
mvp = MeanVariancePortfolio(mu, cov_matrix)

# Put in gamma, max positions. If you want to see the logs, set verbose = True
mp = mvp.efficient_portfolio(gamma, max_positions=K, verbose = False)

print(f"The portfolio's expected return is {round(mp.ret,5)}")

# The portfolio's allocation:
mp.x

The portfolio's expected return is 0.00723


AA    0.000000
BB    0.126590
CC    0.000000
DD    0.470867
EE    0.000000
FF    0.000000
GG    0.000000
HH    0.402543
II    0.000000
JJ    0.000000
dtype: float64

Now it's time to build the same model ourselves. Some parts of the code will use Gurobi's Python matrix API. Here are some [basic examples](https://support.gurobi.com/hc/en-us/articles/17278438215313-Tutorial-Getting-Started-with-the-Gurobi-Python-API).

As a reminder, this is the problem we want to model.

Let $S$ be the set of assets available, $\mu$ is the expected return, and $\Sigma$ is the covariance matrix. $K$ is the max number of assets to invest in. 
$$
\begin{aligned}
\max_{x \in \mathbb{R}^n}\quad 
& \mu^\top x \;-\; \tfrac{\gamma}{2}\, x^\top \Sigma\, x \\
\text{s.t.}\quad 
& \sum_{i \in S} x_i = 1,\\
& x_i \le b_i, \quad \forall i \in S \\
& x_i \ge l*b_i, \quad \forall i \in S \\
& \sum_{i \in S} b_i \le K,\\
& 0 \le x_i \le 1, \quad \forall i \in S
\end{aligned}
$$

In [61]:
#### Now model it yourself
gamma = 20
l = 0.01
# Create an empty optimization model
m = gp.Model()

# Add variables: x[i] denotes the proportion invested in stock i
# 0 <= x[i] <= 1
x = m.addMVar(len(mu), lb=0, ub=1, name="x")
b = m.addMVar(len(mu), vtype=gp.GRB.BINARY, name="b")

# Budget constraint: all investments sum up to 1
m.addConstr(x.sum() == 1, name="Budget_Constraint")

# Link the continuous x to the binary b
# If x > 0 then b = 1 (also remember the bounds of x)
m.addConstr(x <= b, name="Indicator")

# Minimal position; see formula (2) above
m.addConstr(x >= l * b, name="Minimal_Position")
# Cardinality constraint: at most K positions
cardinality_constr = m.addConstr(b.sum() <= 3, "Cardinality")

# Define objective function: Maximize expected utility
m.setObjective(
    mu.to_numpy() @ x - gamma / 2 * (x @ cov_matrix.to_numpy() @ x), gp.GRB.MAXIMIZE
)
m.optimize()

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[rosetta2] - Darwin 24.6.0 24G325)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 22 rows, 20 columns and 60 nonzeros (Max)
Model fingerprint: 0x1d83aebc
Model has 10 linear objective coefficients
Model has 55 quadratic objective terms
Variable types: 10 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [7e-04, 1e-02]
  QObjective range [3e-04, 7e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve time: 0.00s
Presolved: 32 rows, 25 columns, 85 nonzeros
Presolved model has 60 quadratic objective terms
Variable types: 15 continuous, 10 integer (10 binary)
Found heuristic solution: objective -0.0122823
Found heuristic solution: objective -0.0009564
Found heuristic solution: objective 0.0000923

Root relaxation: objective 3.301807e-03, 39 iterations, 0.00 seconds (0.00 w

In [62]:
s2 = pd.Series(x.X, index= mp.x.index)
s1 = pd.Series(mp.x)
df = pd.concat([s2,s1], axis=1, ignore_index=True)
df.columns = ['OptiMod','Our Model']
df

,OptiMod,Our Model
AA,0.000000,0.000000
BB,0.126590,0.126590
CC,0.000000,0.000000
DD,0.470867,0.470867
EE,0.000000,0.000000
FF,0.000000,0.000000
GG,0.000000,0.000000
HH,0.402543,0.402543
II,0.000000,0.000000
JJ,0.000000,0.000000


### Move risk to a constraint
In the models above, the objective represented a utility function that balanced risk and returns. Let's make a couple of changes to the model. 

- Model the risk as constraint, where the objective is now just to maximize return.
- Incorporate fixed and variable transaction costs. 
- Remove the cardinality constraints from before 

Write the formulation where:
$$
\begin{aligned}
&\sigma_0 \space\text{maximal variance for the portfolio} \\
&c  \space\text{is the fixed transaction costs for any asset, relative to total investment value} \\
&l  \space\text{is the lower bound on position size} \\
&f_i \space\text{is the variable transaction fee for each asset relative to total investment value} 
\end{aligned}
$$

$$
\begin{aligned}
\max_{x \in \mathbb{R}^n}\quad &\mu^\top x\\
\text{s.t.}\quad 
&x^\top \Sigma\, x \le \sigma_0\\
& \sum_{i \in S} x_i + c\sum_{i \in S} b_i + \sum_{i \in S} f_ix_i= 1\\
& x_i \le b_i, \quad \forall i \in S \\
& x_i \ge l*b_i, \quad \forall i \in S \\
& 0 \le x_i \le 1 \quad \forall i \in S \\

& b_i \in \{0,1\} \quad \forall i \in S
\end{aligned}
$$

Use the data below to write code for the above model.

In [63]:
data = load_portfolio()
cov_matrix = data.cov()
mu = data.mean()

# Values for the model parameters:
V = 0.0025  # Maximal admissible variance (sigma^2)
l = 0.001  # Minimal position size
c = 0.0001  # Fixed transaction costs
f = 0.001 * np.ones(mu.shape)  # Variable transaction fees

In [64]:
# Create an empty optimization model
m = gp.Model()

# Add variables: x[i] denotes the proportion invested in stock i
x = m.addMVar(len(mu), lb=0, ub=1, name="x")
# Add variables: b[i]=1 if stock i is held, and b[i]=0 otherwise
b = m.addMVar(len(mu), vtype=gp.GRB.BINARY, name="b")
# Upper bound on variance
m.addConstr(x @ cov_matrix.to_numpy() @ x <= V, name="Variance")
# Force x to 0 if not traded; see formula (1) above
m.addConstr(x <= b, name="Indicator")
# Minimal position; see formula (2) above
m.addConstr(x >= l * b, name="Minimal_Position")
# Budget constraint: all investments, costs, and fees sum up to 1
budget_constr = m.addConstr(
    x.sum() + c*b.sum() + f @ x == 1, name="Budget_Constraint"
)
# Define objective: Maximize expected return
m.setObjective(mu.to_numpy() @ x, gp.GRB.MAXIMIZE)

m.optimize()

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[rosetta2] - Darwin 24.6.0 24G325)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 21 rows, 20 columns and 60 nonzeros (Max)
Model fingerprint: 0x3d88eddd
Model has 10 linear objective coefficients
Model has 1 quadratic constraint
Variable types: 10 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  QMatrix range    [1e-05, 3e-03]
  Objective range  [7e-04, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [3e-03, 3e-03]
Presolve time: 0.00s
Presolved: 21 rows, 20 columns, 60 nonzeros
Presolved model has 1 quadratic constraint(s)
Variable types: 10 continuous, 10 integer (10 binary)

Root relaxation: objective 1.390322e-02, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI

In [65]:
pd.Series(x.X)

0    0.000000
1    0.894386
2    0.000000
3    0.104415
4    0.000000
5    0.000000
6    0.000000
7    0.000000
8    0.000000
9    0.000000
dtype: float64

## Interpreting Log Files

### MIPFocus Parameter Analysis

#### Introduction

This section looks at the impact of the **MIPFocus** parameter on solving the `markshare_4_0.mps` model. You can find many models to tinker with in the [MIPLIB - Mixed-Integer Programming Library](https://miplib.zib.de/index.html)

##### What is MIPFocus?

The MIPFocus parameter allows you to modify the high-level solution strategy in Gurobi's MIP solver:

- **MIPFocus = 0** (Default): Balances finding new feasible solutions and proving optimality
- **MIPFocus = 1**: Focuses more on finding good quality feasible solutions quickly
- **MIPFocus = 2**: Focuses more attention on proving optimality (assumes finding solutions is easy)
- **MIPFocus = 3**: Focuses on improving the best objective bound

In this analysis, we'll compare MIPFocus values 0, 1, and 2 on the glass4 model.

##### Metrics to Analyze

We will compare:
- Solution time
- Number of nodes explored
- Number of simplex iterations
- Final optimality gap
- Objective value
- Solution status

In [66]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import time

#### Helper Function to Solve and Collect Statistics

We'll create a helper function that:
1. Loads the model
2. Sets the MIPFocus parameter
3. Solves the model
4. Collects performance metrics

<div style="
  border-left: 6px solid #ef4444;  /* red */
  padding:14px 16px;
  border-radius:10px;
  margin:12px 0;
  font-size:16px; line-height:1.4;">
  <strong style="font-size:18px; color:#b91c1c;">❗ IMPORTANT</strong><br>
  Download the mps file and unzip/extract it. <br>
  You can find the file here:
  <a href="https://miplib.zib.de/instance_details_markshare_4_0.html" target="_blank" rel="noopener noreferrer">
    MIPLIB — markshare_4_0 instance page
  </a>.
  <br><br>
  Once you do this, if you downloaded this notebook and are running this locally make sure the file is in the same folder. <br>
  If you are in Colab, click on the folder on the left part of your screen. Then drag the mps file there.
</div>


In [ ]:
mps_file = 'markshare_4_0.mps' 

In [ ]:
def solve_with_mipfocus(model_file = mps_file, mipfocus_value = 0, output_flag = 1, time_limit=60):
    """
    Solve a model with a specific MIPFocus value and return performance metrics.
    
    Parameters:
    -----------
    model_file : str
        Path to the .mps file
    mipfocus_value : int
        MIPFocus parameter value (0, 1, 2, or 3)
    time_limit : float
        Time limit in seconds
    
    Returns:
    --------
    dict : Dictionary containing performance metrics
    """
    # Create environment and model
    # This will make the name of a '*.log' file
    log_file = f'mipfocus{mipfocus_value}.log'
    env = gp.Env(params={"LogFile": log_file, 
                         "OutputFlag": output_flag, 
                         "LogToConsole": 0})  
    model = gp.read(model_file, env=env)
    
    # Set parameters
    model.setParam('MIPFocus', mipfocus_value)
    model.setParam('TimeLimit', time_limit)
    
    print(f"\n{'='*80}")
    print(f"Solving with MIPFocus = {mipfocus_value}")
    print(f"{'='*80}\n")
    
    # Solve the model
    model.optimize()
    
    # Collect metrics
    metrics = {
        'MIPFocus': mipfocus_value,
        'Status': model.Status,
        'StatusName': model.getAttr(GRB.Attr.Status),
        'Runtime': model.Runtime,
        'NodeCount': model.NodeCount,
        'IterCount': model.IterCount,
    }
    
    # Add solution-specific metrics if available
    if model.SolCount > 0:
        metrics['ObjVal'] = model.ObjVal
        metrics['ObjBound'] = model.ObjBound
        metrics['MIPGap'] = model.MIPGap
        metrics['SolCount'] = model.SolCount
    else:
        metrics['ObjVal'] = None
        metrics['ObjBound'] = model.ObjBound if hasattr(model, 'ObjBound') else None
        metrics['MIPGap'] = None
        metrics['SolCount'] = 0

    
    # Clean up
    model.dispose()
    env.dispose()
    
    return metrics

#### MIPFocus = 0 (Default - Balanced Approach)

The default MIPFocus setting strikes a balance between finding new feasible solutions and proving optimality. This is typically where you want to start.

In [69]:
# Solve with MIPFocus = 0
results_0 = solve_with_mipfocus(mipfocus_value=0)

for key, value in results_0.items():
    if key != 'StatusName':
        print(f"{key:20s}: {value}")


Solving with MIPFocus = 0

MIPFocus            : 0
Status              : 2
Runtime             : 0.2773580551147461
NodeCount           : 5816.0
IterCount           : 10464.868172922967
ObjVal              : 1.0
ObjBound            : 1.0
MIPGap              : 0.0
SolCount            : 10


#### MIPFocus = 1 (Focus on Feasibility)

With MIPFocus = 1, this focuses more on finding good feasible solutions quickly. This is useful when:
- You need solutions more quickly
- Proving optimality is less critical
- The model is difficult and finding any feasible solution is challenging

In [70]:
# Solve with MIPFocus = 1
results_1 = solve_with_mipfocus(mipfocus_value=1)

for key, value in results_1.items():
    if key != 'StatusName':
        print(f"{key:20s}: {value}")


Solving with MIPFocus = 1

MIPFocus            : 1
Status              : 2
Runtime             : 0.4498732089996338
NodeCount           : 6744.0
IterCount           : 11554.081539166284
ObjVal              : 1.0
ObjBound            : 1.0
MIPGap              : 0.0
SolCount            : 10


#### MIPFocus = 2 (Focus on Optimality)

With MIPFocus = 2, the focus is more on proving optimality. Try this when:
- The solver finds good solutions quickly
- The optimality gap closes slowly

In [71]:
# Solve with MIPFocus = 2
results_2 = solve_with_mipfocus(mipfocus_value=2)

for key, value in results_2.items():
    if key != 'StatusName':
        print(f"{key:20s}: {value}")


Solving with MIPFocus = 2

MIPFocus            : 2
Status              : 2
Runtime             : 0.23417115211486816
NodeCount           : 9766.0
IterCount           : 16301.495768467246
ObjVal              : 1.0
ObjBound            : 1.0
MIPGap              : 0.0
SolCount            : 10


#### MIPFocus = 3 (Focus on Bound)

With MIPFocus = 3, we focus on improving the best objective bound. So try when:

- The best objective bound is moving very slowly or not at all
- You want to make progress on the dual bound (lower bound for minimization, upper bound for maximization)
- The solver is struggling to prove optimality because the bound improvement has stalled

In [72]:
# Solve with MIPFocus = 3
results_3 = solve_with_mipfocus(mipfocus_value=3)

for key, value in results_3.items():
    if key != 'StatusName':
        print(f"{key:20s}: {value}")


Solving with MIPFocus = 3

MIPFocus            : 3
Status              : 2
Runtime             : 0.19624996185302734
NodeCount           : 4358.0
IterCount           : 8187.345183761711
ObjVal              : 1.0
ObjBound            : 1.0
MIPGap              : 0.0
SolCount            : 10


#### Comparative Analysis

Now let's compare the results across each of the four MIPFocus values to understand their impact on the solve.

In [73]:
# Create a comparison dataframe
comparison_df = pd.DataFrame([results_0, results_1, results_2, results_3])
comparison_df = comparison_df.drop('StatusName', axis=1)
comparison_df

,MIPFocus,Status,Runtime,NodeCount,IterCount,ObjVal,ObjBound,MIPGap,SolCount
0,0,2,0.277358,5816.0,10464.868173,1.0,1.0,0.0,10
1,1,2,0.449873,6744.0,11554.081539,1.0,1.0,0.0,10
2,2,2,0.234171,9766.0,16301.495768,1.0,1.0,0.0,10
3,3,2,0.196250,4358.0,8187.345184,1.0,1.0,0.0,10


<div style="
  border-left: 6px solid #3b82f6;  /* blue */
  padding:14px 16px;
  border-radius:10px;
  margin:12px 0;
  font-size:16px; line-height:1.4;">
  <strong style="font-size:18px;">🧠 Explore the logs!</strong><br>
  Each log file should have saved where you have the mps file. Open them up and start to compare. 
  
  For example, which value of MIPFocus found a better solution faster?
</div>

### Quadratic assignment problem

Here is a somewhat random variation of the `quadratic assignment problem`. Here is a formulation, but we really won't need it but it's here for fun. 

#### Mathematical Formulation

##### Sets and Indices
- $i, k \in \{0, 1, \ldots, n-1\}$: Facilities
- $j, l \in \{0, 1, \ldots, n-1\}$: Locations

##### Parameters
- $f_{ik}$: Flow between facility $i$ and facility $k$ (represented as `flow[i][k]`)
- $d_{jl}$: Distance between location $j$ and location $l$ (represented as `dist[j][l]`)
- $n$: Number of facilities (and locations)

##### Decision Variables
- $x_{ij} \in \{0, 1\}$: Binary variable equal to 1 if facility $i$ is assigned to location $j$, 0 otherwise

##### Objective Function

$$
\min \sum_{i=0}^{n-1} \sum_{j=0}^{n-1} \sum_{k=0}^{n-1} \sum_{l=0}^{n-1} c_{ik} \cdot f_{ik} \cdot d_{jl} \cdot x_{ij} \cdot x_{kl}
$$

where the coefficient $c_{ik}$ is defined as:

$$
c_{ik} = \begin{cases} 
-1 & \text{if } (i+k) \bmod 2 = 0 \\
1 & \text{if } (i+k) \bmod 2 = 1
\end{cases}
$$

##### Constraints

**Assignment constraints - Each facility to exactly one location:**
$$
\sum_{j=0}^{n-1} x_{ij} = 1 \quad \forall i \in \{0, 1, \ldots, n-1\}
$$

**Assignment constraints - Each location receives exactly one facility:**
$$
\sum_{i=0}^{n-1} x_{ij} = 1 \quad \forall j \in \{0, 1, \ldots, n-1\}
$$

**Binary constraints:**
$$
x_{ij} \in \{0, 1\} \quad \forall i, j \in \{0, 1, \ldots, n-1\}
$$

Use this data and the model below to solve the problem. You may notice it take a little while. 

In [82]:
n = 10

# Distance between locations
dist = [[abs(j - k) + 1 for k in range(n)] for j in range(n)]

# Flow/weight between facilities
flow = [[(i + k + 1) * ((i - k) ** 2 + 1) for k in range(n)] for i in range(n)]

Here is the model. You shouldn't have to edit anything now. 

In [83]:
# Quadratic assignment problem
m1 = gp.Model()

# x[i,j] = 1 if facility i is assigned to location j
x = m1.addVars(n, n, vtype=GRB.BINARY, name="x")

# Build objective function
obj = gp.QuadExpr()
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                coeff = flow[i][k] * dist[j][l]
                # Alternate signs
                if (i + k) % 2 == 0:
                    coeff = -coeff
                obj.addTerms(coeff, x[i, j], x[k, l])

m1.setObjective(obj, GRB.MINIMIZE)

# Each facility is assigned to exactly one location
for i in range(n):
    m1.addConstr(x.sum(i, "*") == 1, name=f"assign_facility[{i}]")

# Each location is assigned exactly one facility
for j in range(n):
    m1.addConstr(x.sum("*", j) == 1, name=f"assign_to_location[{j}]")

Run this without editing any parameters to see how long it takes. Then, try to recall a parameter mentioned in the log file session that may fit this type of problem. Try searching the [documentation](https://docs.gurobi.com/projects/optimizer/en/current/).

- Hint 1: Look at the type of terms you see in the log file and think about how these terms are typically dealt with. 
- Hint 2: We tend to like things *linear*.
- Hint 3: You'll be told what it is in a couple of minutes. 

In [84]:
# Set parameters here
# m.Params.ParameterName = value
m1.optimize()

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[rosetta2] - Darwin 24.6.0 24G325)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 20 rows, 100 columns and 200 nonzeros (Min)
Model fingerprint: 0xb35f04ea
Model has 0 linear objective coefficients
Model has 5050 quadratic objective terms
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1644.0000000
Presolve time: 0.02s
Presolved: 20 rows, 100 columns, 200 nonzeros
Presolved model has 5050 quadratic objective terms
Variable types: 0 continuous, 100 integer (100 binary)

Root relaxation: objective -6.799610e+05, 119 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     

Here's the whole model with the updated parameter. 

In [77]:
n = 9

# Quadratic assignment problem
m2 = gp.Model()
# Distance between locations
dist = [[abs(j - k) + 1 for k in range(n)] for j in range(n)]

# Flow/weight between facilities
flow = [[(i + k + 1) * ((i - k) ** 2 + 1) for k in range(n)] for i in range(n)]

# x[i,j] = 1 if facility i is assigned to location j
x = m2.addVars(n, n, vtype=GRB.BINARY, name="x")

# Build objective function
obj = gp.QuadExpr()
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                coeff = flow[i][k] * dist[j][l]
                # Alternate signs
                if (i + k) % 2 == 0:
                    coeff = -coeff
                obj.addTerms(coeff, x[i, j], x[k, l])

m2.setObjective(obj, GRB.MINIMIZE)

# Each facility is assigned to exactly one location
for i in range(n):
    m2.addConstr(x.sum(i, "*") == 1, name=f"assign_facility[{i}]")

# Each location is assigned exactly one facility
for j in range(n):
    m2.addConstr(x.sum("*", j) == 1, name=f"assign_to_location[{j}]")

# Set parameters here
m2.Params.PreQLinearize = 2

m2.optimize()

Set parameter PreQLinearize to value 2
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[rosetta2] - Darwin 24.6.0 24G325)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
PreQLinearize  2

Optimize a model with 18 rows, 81 columns and 162 nonzeros (Min)
Model fingerprint: 0x05cc9672
Model has 0 linear objective coefficients
Model has 3321 quadratic objective terms
Variable types: 0 continuous, 81 integer (81 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 765.0000000
Presolve time: 0.00s
Presolved: 98 rows, 161 columns, 3562 nonzeros
Variable types: 0 continuous, 161 integer (82 binary)

Root relaxation: objective -2.007734e+05, 116 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Ob